# Notebook to build a *Lipomyces Starkeyi* draft GSM 



The *Lipomyces starkeyi* draft model (Lst) is built using *Rhodosporidium* IFO0880 (rto) as a scaffold. The process for building the model is as follows:

* Map the genes from IFO0880 to Lst (using NCBI command line blast tool to score best matches).
* Update the Lst model
* Perform manual curation of simulations
* Check performance

This is the a barebones model that is used as a framework for building the complete model. 

#### import necessary libraries

In [1]:
import pandas as pd
import cobra
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import gapfill

## Gene Mapping 

Using blastp results from the Lst and IFO0880 genomes (comparing both Lst to IFO0880 and IFO0880 to Lst).
blastp results compiled via Joonhoon and deposited into text files referenced below.

### Load blastp results

In [2]:
blast_header = ['qseqid','sseqid','pident','length','mismatch','gapopen',
                'qstart','qend','sstart','send','evalue','bitscore']

Lst to IFO0880 

In [3]:
Lst_to_IFO0880_4 = pd.read_table('../blastp/Lipst1_1_to_IFO0880_4.txt', index_col=0, names=blast_header)
Lst_to_IFO0880_4.head()
Lst_to_IFO0880_4_full = Lst_to_IFO0880_4.copy()

In [4]:
# Lst_to_IFO0880_4.to_csv('lipst1_1_toIFO0880_4.csv')

Filter based on the 'evalue' result returned

In [5]:
Lst_to_IFO0880_4 = Lst_to_IFO0880_4[Lst_to_IFO0880_4['evalue'] < 1e-6]
print(len(Lst_to_IFO0880_4))
Lst_to_IFO0880_4.head()

33640


,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
qseqid,,,,,,,,,,,
Lipst1_1_89131,9275,42.857,196,95,7,9,189,42,235,8.030000e-41,139.0
Lipst1_1_89131,11708,36.019,211,122,4,12,209,13,223,5.230000e-29,112.0
Lipst1_1_89131,11825,33.032,221,127,6,28,227,125,345,7.020000e-29,110.0
Lipst1_1_89131,11824,35.938,192,106,7,1,181,92,277,7.630000e-26,102.0
Lipst1_1_89131,12324,27.179,195,119,4,11,182,75,269,8.050000e-22,91.7


IFO0880 to Lst 

In [6]:
IFO0880_4_to_Lst = pd.read_table('../blastp/IFO0880_4_to_Lipst1_1.txt', index_col=0, names=blast_header, )
IFO0880_4_to_Lst.index = IFO0880_4_to_Lst.index.astype('str')
IFO0880_4_to_Lst.head()

,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
qseqid,,,,,,,,,,,
10000,Lipst1_1_75163,63.294,425,149,3,1,422,86,506,0.0,549.0
10000,Lipst1_1_92653,40.541,37,21,1,328,364,864,899,1.7,29.3
10000,Lipst1_1_4357,27.049,122,71,4,242,358,115,223,4.6,27.7
10000,Lipst1_1_338855,32.000,50,30,1,133,178,17,66,4.8,26.9
10000,Lipst1_1_6497,32.432,37,25,0,136,172,386,422,6.6,26.9


Filter based on the 'evalue' result returned

In [7]:
IFO0880_4_to_Lst = IFO0880_4_to_Lst[IFO0880_4_to_Lst['evalue'] < 1e-6]
print(len(IFO0880_4_to_Lst))
IFO0880_4_to_Lst.head()

33896


,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
qseqid,,,,,,,,,,,
10000,Lipst1_1_75163,63.294,425,149,3,1,422,86,506,0.000000e+00,549.0
10001,Lipst1_1_5601,33.180,434,279,6,85,512,12,440,1.660000e-59,201.0
10001,Lipst1_1_319429,32.409,469,287,11,64,512,73,531,3.830000e-57,197.0
10001,Lipst1_1_48163,32.471,425,267,6,98,513,24,437,7.140000e-56,192.0
10001,Lipst1_1_72166,29.955,444,297,8,77,512,26,463,4.590000e-53,185.0


### Map best hits 

ussing bitscore

In [8]:
IFO0880_4_to_Lst_map = dict()
for k in IFO0880_4_to_Lst.index.unique():
    temp = IFO0880_4_to_Lst.loc[[k]]
    IFO0880_4_to_Lst_map[k] = temp.loc[temp['bitscore'] > temp['bitscore'].max()*0.95,'sseqid'].tolist()

In [9]:
print(len(IFO0880_4_to_Lst_map.keys()))
print(len(sum([x for x in IFO0880_4_to_Lst_map.values()],[])))
print(len(set(sum([x for x in IFO0880_4_to_Lst_map.values()],[]))))

4825
5340
4009


In [10]:
Lst_to_IFO0880_4_map = dict()
for k in Lst_to_IFO0880_4.index.unique():
    temp = Lst_to_IFO0880_4.loc[[k]]
    Lst_to_IFO0880_4_map[k] = temp.loc[temp['bitscore'] > temp['bitscore'].max()*0.95,'sseqid'].astype('str').tolist()

In [11]:
print(len(Lst_to_IFO0880_4_map.keys()))
print(len(sum([x for x in Lst_to_IFO0880_4_map.values()],[])))
print(len(set(sum([x for x in Lst_to_IFO0880_4_map.values()],[]))))

4847
5296
3913


### Map bidirectional best hits

In [12]:
BBH_map = dict()
for k, v in IFO0880_4_to_Lst_map.items():
    temp = set(x for x in v if x in Lst_to_IFO0880_4_map and k in Lst_to_IFO0880_4_map[x])
    if temp:
        BBH_map[k] = temp

In [13]:
BBH_map_strict = BBH_map.copy()

In [14]:
print(len(BBH_map.keys()))
print(len(sum([list(x) for x in BBH_map.values()],[])))
print(len(set(sum([list(x) for x in BBH_map.values()],[]))))

3780
3924
3822


In [15]:
# BBH_map

### Load R. toruloides model and build a draft model

In [15]:
rto = cobra.io.load_json_model('../models/Rt_IFO0880.json')
rto

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"


Name,Rt_IFO0880
Memory address,2953f04d0
Number of metabolites,2051
Number of reactions,2398
Number of genes,1142
Number of groups,0
Objective expression,1.0*BIOMASS_RT - 1.0*BIOMASS_RT_reverse_2b3e0
Compartments,"c, x, m, e, r, v, n, g, d"


In [16]:
rto.medium

{'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_glc__D_e': 1.0,
 'EX_ca2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [17]:
rto.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,6.104E-05,0,0.00%
cu2_e,EX_cu2_e,3.852E-05,0,0.00%
fe3_e,EX_fe3_e,0.0004526,0,0.00%
glc__D_e,EX_glc__D_e,1,6,98.64%
k_e,EX_k_e,0.04297,0,0.00%
mg2_e,EX_mg2_e,0.004528,0,0.00%
mn2_e,EX_mn2_e,4.453E-05,0,0.00%
na1_e,EX_na1_e,0.001915,0,0.00%
nh4_e,EX_nh4_e,0.4842,0,0.00%
o2_e,EX_o2_e,2.287,0,0.00%


#### obtain list of rto reactions not mapped to a lst gene

In [18]:
reactions_not_mapped_rto = set()
for r in rto.reactions:
    if r.genes and not any(g.id in BBH_map for g in r.genes):
        print(r)
        reactions_not_mapped_rto.add(r.id)

FACOAL80p: atp_x + coa_x + octa_x --> amp_x + occoa_x + ppi_x
URIH: h2o_c + uri_c --> rib__D_c + ura_c
ACACT4m: accoa_m + occoa_m <-- 3odcoa_m + coa_m
CHTNDA: chitin_c + h2o_c --> ac_c + chitos_c + h_c
ACOAD8m: fad_m + ivcoa_m --> 3mb2coa_m + fadh2_m
CYTD: cytd_c + h2o_c + h_c --> nh4_c + uri_c
DPGM: 13dpg_c <=> 23dpg_c + h_c
LEUTA: akg_c + leu__L_c <=> 4mop_c + glu__L_c
FA140COAabcp: atp_c + 2.0 h2o_c + tdcoa_c --> adp_c + coa_c + 2.0 h_c + pi_c + ttdca_x
MCITSm: h2o_m + oaa_m + ppcoa_m --> 2mcit_m + coa_m + h_m
GNNUC: gsn_c + h2o_c --> gua_c + rib__D_c
FA180COAabcp: atp_c + 2.0 h2o_c + stcoa_c --> adp_c + coa_c + 2.0 h_c + ocdca_x + pi_c
H2Otm: h2o_c <=> h2o_m
SACCD4m: h2o_m + nadp_m + saccrp__L_m --> L2aadp6sa_m + glu__L_m + h_m + nadph_m
ADNUC: adn_c + h2o_c --> ade_c + rib__D_c
FA182COAabcp: atp_c + 2.0 h2o_c + ocdycacoa_c --> adp_c + coa_c + 2.0 h_c + ocdcya_x + pi_c
ACOAH: ac_c + coa_c + h_c <-- accoa_c + h2o_c
NNAMrm: h2o_m + ncam_m <=> nac_m + nh4_m
GTMLT: ala__L_c + gthrd_c -

#### build draft lst model from rto

In [19]:
model = cobra.Model()
for r in rto.reactions:
    if not r.genes:
        model.add_reactions([r.copy()])
    elif any(g.id in BBH_map for g in r.genes):
        r_copy = r.copy()
        temp_gpr = r.gene_reaction_rule
        for g in r.genes:
            if g.id in BBH_map:
                if len(BBH_map[g.id]) == 1:
                    temp_gpr = temp_gpr.replace(g.id, list(BBH_map[g.id])[0])
                else:
                    temp_gpr = temp_gpr.replace(g.id, '('+' or '.join(BBH_map[g.id])+')')
        r_copy.gene_reaction_rule = temp_gpr
        model.add_reactions([r_copy])

#### Test the draft model for growth

In [20]:
model.reactions.get_by_id('BIOMASS_RT').id = 'BIOMASS_Ls'
model.objective = 'BIOMASS_Ls'
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,0.732,6,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-0.732,2,33.33%
btd_RR_e,EX_btd_RR_e,-0.244,4,22.22%
co2_e,EX_co2_e,-0.488,1,11.11%
h_e,EX_h_e,-0.732,0,0.00%
sbt__D_e,EX_sbt__D_e,-0.244,6,33.33%


In [21]:
model.optimize()

,fluxes,reduced_costs
ALCD25yi,0.0,0.0
MTHFCm,0.0,0.0
AMPN,0.0,0.0
DAGCPTer_RT,0.0,0.0
PYRt2,0.0,0.0
...,...,...
FOLt,0.0,0.0
NADtm,0.0,0.0
EX_pydxn_e,0.0,0.0
PYDXNtr,0.0,0.0


#### The draft model cannot simulate growth

In [22]:
metabolites_not_synthesized_original = set()
for m, v in model.reactions.BIOMASS_Ls.metabolites.items():
    if v < 0:
        with model:
#             for x in ['ALDD2ym','C30CPT1','C3STKR1er','C3STKR2er','DNTPPA','DRTPPD','PMDPHT']:
#                 r = rto.reactions.get_by_id(x).copy()
#                 model_rto_based_v2.add_reaction(r)
            temp = model.add_boundary(m, type='demand')
            model.objective = temp.id
            sol = model.optimize()
            if not sol.objective_value:
#                 print(m)
                metabolites_not_synthesized_original.add(m.id)
len(metabolites_not_synthesized_original)
                

30

# Examining models. 

In [23]:
model

Name,None
Memory address,296550f90
Number of metabolites,1897
Number of reactions,2104
Number of genes,1069
Number of groups,0
Objective expression,1.0*BIOMASS_Ls - 1.0*BIOMASS_Ls_reverse_27d85
Compartments,"c, m, r, e, x, v, n, g, d"


In [25]:
gapfiller = cobra.flux_analysis.gapfilling.GapFiller(model, rto, demand_reactions=False, integer_threshold=1e-9)
gapfiller.model.solver.configuration.tolerances.feasibility = 1e-9
gapfiller.model.solver.configuration.tolerances.integrality = 1e-10
# gapfiller.model.solver.configuration.tolerances.optimality = 1e-9
gapfiller.fill()

[[<Reaction H2Otm at 0x29ee50990>,
  <Reaction DRTPPD at 0x29eeade10>,
  <Reaction ALDD2ym at 0x29efe1910>,
  <Reaction DNTPPA at 0x29f171e50>,
  <Reaction C3STKR1er at 0x29efa92d0>,
  <Reaction C3STKR2er at 0x29f189210>,
  <Reaction PMDPHT at 0x29f194590>,
  <Reaction C30CPT1 at 0x29f197ed0>,
  <Reaction ACRNtp at 0x29f1a4510>,
  <Reaction FA183COAabcp at 0x29f1a6150>,
  <Reaction CRNtp at 0x29f23b210>]]

In [24]:
growing_model = model.copy()

for i in ['H2Otm','DRTPPD','ALDD2ym','DNTPPA',
          'C3STKR1er','C3STKR2er','PMDPHT','C30CPT1',
          'ACRNtp','FA183COAabcp','CRNtp']:
    r = rto.reactions.get_by_id(i)
    growing_model.add_reactions([r.copy()])

Read LP format model from file /var/folders/kn/zzns_smn1q79xdnf1tgqk__r0000gn/T/tmp_ihgsnrh.lp
Reading time = 0.01 seconds
: 1897 rows, 4208 columns, 17198 nonzeros


In [25]:
growing_model.optimize()

,fluxes,reduced_costs
ALCD25yi,0.000000,-1.110223e-16
MTHFCm,0.000000,-0.000000e+00
AMPN,0.000000,-2.990537e-03
DAGCPTer_RT,0.000000,0.000000e+00
PYRt2,0.000000,0.000000e+00
...,...,...
PMDPHT,0.000105,0.000000e+00
C30CPT1,-0.000258,-0.000000e+00
ACRNtp,0.111393,0.000000e+00
FA183COAabcp,0.012377,0.000000e+00


### removing Rto genes from L. starkeyi model. 

In [26]:
model_removed = model.copy()
cobra.manipulation.remove_genes(model_removed, [g.id for g in model.genes if not g.id.startswith('Lipst1')])

Read LP format model from file /var/folders/kn/zzns_smn1q79xdnf1tgqk__r0000gn/T/tmphmm3gh7w.lp
Reading time = 0.01 seconds
: 1897 rows, 4208 columns, 17198 nonzeros


In [27]:
model_removed.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,0.732,6,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-0.732,2,33.33%
btd_RR_e,EX_btd_RR_e,-0.244,4,22.22%
co2_e,EX_co2_e,-0.488,1,11.11%
h_e,EX_h_e,-0.732,0,0.00%
sbt__D_e,EX_sbt__D_e,-0.244,6,33.33%


In [28]:
model_removed

Name,None
Memory address,2a3a6ff50
Number of metabolites,1897
Number of reactions,2062
Number of genes,895
Number of groups,0
Objective expression,1.0*BIOMASS_Ls - 1.0*BIOMASS_Ls_reverse_27d85
Compartments,"c, m, r, e, x, v, n, g, d"


Finding reactions needed for growth. 

#### Reactions removed by removing R. toruloides genes 

In [29]:
reactions_needing_genes = []
for r in growing_model.reactions:
    if r.id not in model_removed.reactions:
        reactions_needing_genes.append(r.id)
        print(r.id, r.gene_reaction_rule)

CYOR_u9m COB and Lipst1_1_72848 and Lipst1_1_112667 and Lipst1_1_38038 and Lipst1_1_47735 and Lipst1_1_3376 and Lipst1_1_42388 and 15231 and Lipst1_1_6808 and Lipst1_1_70538 and Lipst1_1_65274 and Lipst1_1_49747
FAO141p_even (10293 and Lipst1_1_75883 and Lipst1_1_158150 and 12742 and Lipst1_1_160699 and Lipst1_1_47324) or (10293 and Lipst1_1_75883 and Lipst1_1_158150 and 12752 and Lipst1_1_160699 and Lipst1_1_47324) or (10293 and Lipst1_1_75883 and Lipst1_1_158150 and Lipst1_1_160699 and Lipst1_1_47324 and Lipst1_1_2790)
RNDR3n (10848 and Lipst1_1_2363 and Lipst1_1_29458 and Lipst1_1_2363) or (Lipst1_1_2363 and Lipst1_1_29458 and 12730 and Lipst1_1_2363) or (Lipst1_1_2363 and Lipst1_1_29458 and 12737 and Lipst1_1_2363)
FAO161p_odd (10293 and Lipst1_1_75883 and 12742 and Lipst1_1_47324) or (10293 and Lipst1_1_75883 and 12752 and Lipst1_1_47324) or (10293 and Lipst1_1_75883 and Lipst1_1_47324 and Lipst1_1_2790)
FAO161p_even (10293 and Lipst1_1_75883 and Lipst1_1_158150 and 12742 and Lips

In [30]:
len(reactions_needing_genes)

53

#### gap fill model with rto. 

In [31]:
model_removed.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,0.732,6,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-0.732,2,33.33%
btd_RR_e,EX_btd_RR_e,-0.244,4,22.22%
co2_e,EX_co2_e,-0.488,1,11.11%
h_e,EX_h_e,-0.732,0,0.00%
sbt__D_e,EX_sbt__D_e,-0.244,6,33.33%


In [32]:
growing_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,5.024E-05,0,0.00%
cu2_e,EX_cu2_e,3.17E-05,0,0.00%
fe3_e,EX_fe3_e,0.0003725,0,0.00%
glc__D_e,EX_glc__D_e,1,6,98.88%
k_e,EX_k_e,0.03536,0,0.00%
mg2_e,EX_mg2_e,0.003726,0,0.00%
mn2_e,EX_mn2_e,3.665E-05,0,0.00%
na1_e,EX_na1_e,0.001576,0,0.00%
nh4_e,EX_nh4_e,0.38,0,0.00%
o2_e,EX_o2_e,2.35,0,0.00%


In [33]:
rto.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,6.104E-05,0,0.00%
cu2_e,EX_cu2_e,3.852E-05,0,0.00%
fe3_e,EX_fe3_e,0.0004526,0,0.00%
glc__D_e,EX_glc__D_e,1,6,98.64%
k_e,EX_k_e,0.04297,0,0.00%
mg2_e,EX_mg2_e,0.004528,0,0.00%
mn2_e,EX_mn2_e,4.453E-05,0,0.00%
na1_e,EX_na1_e,0.001915,0,0.00%
nh4_e,EX_nh4_e,0.4842,0,0.00%
o2_e,EX_o2_e,2.287,0,0.00%


In [34]:
# determine the essential reactions in model 
essential_reactions_model = cobra.flux_analysis.find_essential_reactions(growing_model)

# how many essential reactions are in the model_removed? 
for r in essential_reactions_model:
    
    # print the ones lacking
    if r.id not in model_removed.reactions:
        print(r, r.gene_reaction_rule)

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/kn/zzns_smn1q79xdnf1tgqk__r0000gn/T/tmpf_f_1vcw.lp
Reading time = 0.01 seconds
: 1898 rows, 4230 columns, 17294 nonzeros
Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/kn/zzns_smn1q79xdnf1tgqk__r0000gn/T/tmphpzmf3pn.lp
Reading time = 0.01 seconds
: 1898 rows, 4230 columns, 17294 nonzeros
Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/kn/zzns_smn1q79xdnf1tgqk__r0000gn/T/tmpp8s2y6t4.lp
Reading time = 0.01 seconds
: 1898 rows, 4230 columns, 17294 nonzeros
Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/kn/zzns_smn1q79xdnf1tgqk__r0000gn/T/tmp14pj_jlo.lp
Reading time = 0.01 seconds
: 1898 rows, 4230 columns, 17294 nonzeros
Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/kn/zzn

In [54]:
gapfiller = cobra.flux_analysis.gapfilling.GapFiller(model_removed, growing_model, demand_reactions=False, integer_threshold=1e-12)
gapfiller.model.solver.configuration.tolerances.feasibility = 1e-9
gapfiller.model.solver.configuration.tolerances.integrality = 1e-10
# gapfiller.model.solver.configuration.tolerances.optimality = 1e-9
gapfiller.fill()

[[<Reaction CYOR_u9m at 0x7f21471c4890>,
  <Reaction TRDR at 0x7f2144eb68d0>,
  <Reaction SERPTer at 0x7f2144ed4a50>,
  <Reaction ATPS3m at 0x7f2144e71250>,
  <Reaction PRPPS at 0x7f2144da4a50>,
  <Reaction CYOO6m at 0x7f2144dc2a50>,
  <Reaction OBDHm at 0x7f2144cc0310>,
  <Reaction FAO183p_even at 0x7f2144cc05d0>,
  <Reaction CRNtp at 0x7f2144bcc710>,
  <Reaction H2Otm at 0x7f2144bcc750>,
  <Reaction DRTPPD at 0x7f2144bcc790>,
  <Reaction ALDD2ym at 0x7f2144bcc7d0>,
  <Reaction DNTPPA at 0x7f2144bcc810>,
  <Reaction C3STKR1er at 0x7f2144bcc850>,
  <Reaction C3STKR2er at 0x7f2144bcc890>,
  <Reaction PMDPHT at 0x7f2144bcc8d0>,
  <Reaction C30CPT1 at 0x7f2144bcc910>,
  <Reaction ACRNtp at 0x7f2144bcc950>,
  <Reaction FA183COAabcp at 0x7f2144bcc990>]]

In [56]:
gapfiller = cobra.flux_analysis.gapfilling.GapFiller(model_removed, rto, demand_reactions=False, integer_threshold=1e-10)
gapfiller.model.solver.configuration.tolerances.feasibility = 1e-9
gapfiller.model.solver.configuration.tolerances.integrality = 1e-10
gapfiller.model.solver.configuration.tolerances.optimality = 1e-9
gapfiller.fill()

[[<Reaction CYOR_u9m at 0x7f21359a69d0>,
  <Reaction H2Otm at 0x7f21359a6f90>,
  <Reaction DRTPPD at 0x7f21336d8250>,
  <Reaction TRDR at 0x7f21336d8d50>,
  <Reaction SERPTer at 0x7f21336f4f90>,
  <Reaction ATPS3m at 0x7f2133713790>,
  <Reaction FA240tp at 0x7f213366bb10>,
  <Reaction ALDD2ym at 0x7f2144ef62d0>,
  <Reaction PRPPS at 0x7f2142461a10>,
  <Reaction CYOO6m at 0x7f2158b16510>,
  <Reaction OIVD2m at 0x7f2156755810>,
  <Reaction DNTPPA at 0x7f215600d8d0>,
  <Reaction C3STKR1er at 0x7f2153b97690>,
  <Reaction C3STKR2er at 0x7f2153b978d0>,
  <Reaction PMDPHT at 0x7f2153b97510>,
  <Reaction C30CPT1 at 0x7f215b7894d0>,
  <Reaction ACRNtp at 0x7f215b7891d0>,
  <Reaction CRNtp at 0x7f215b773590>]]

In [35]:
rto.reactions.get_by_id('FA240tp')

Reaction identifier,FA240tp
Name,Fatty acid peroxisomal transport
Memory address,0x2971c0d10
Stoichiometry,ttc_c --> ttc_x Tetracosanoate n C240 C24H47O2 --> Tetracosanoate n C240 C24H47O2
GPR,9912
Lower bound,0.0
Upper bound,1000.0


In [36]:
rto.reactions.get_by_id('FA183COAabcp')

Reaction identifier,FA183COAabcp
Name,Fatty acid peroxisomal transport via ABC system
Memory address,0x2977bd6d0
Stoichiometry,atp_c + 2.0 h2o_c + lnlncgcoa_c --> adp_c + coa_c + 2.0 h_c + lnlncg_x + pi_c ATP C10H12N5O13P3 + 2.0 H2O H2O + Gamma-linolenoyl-CoA --> ADP C10H12N5O10P2 + Coenzyme A + 2.0 H+ + Gamma-linolenic acid + Phosphate
GPR,13167 and 9637
Lower bound,0.0
Upper bound,1000.0


In [37]:
rto.reactions.get_by_id('FA160COAabcp')

Reaction identifier,FA160COAabcp
Name,Fatty acyl CoA peroxisomal transport via ABC system
Memory address,0x297279690
Stoichiometry,atp_c + 2.0 h2o_c + pmtcoa_c --> adp_c + coa_c + 2.0 h_c + hdca_x + pi_c ATP C10H12N5O13P3 + 2.0 H2O H2O + Palmitoyl-CoA (n-C16:0CoA) --> ADP C10H12N5O10P2 + Coenzyme A + 2.0 H+ + Hexadecanoate (n-C16:0) + Phosphate
GPR,13167 and 9637
Lower bound,0.0
Upper bound,1000.0


Note. during original gap filling procedures, FA160COAabcp was obtained instead of FA240tp. FA160COAabcp results in a higher biomass objective (0.0613 vs 0.057).

In [41]:
# all three peroxisome transporters included.

reactions = ['CYOR_u9m','H2Otm','DRTPPD','TRDR','SERPTer','ATPS3m','FA240tp','FA160COAabcp','FA183COAabcp',
            'ALDD2ym','PRPPS','CYOO6m','OIVD2m','DNTPPA','C3STKR1er','C3STKR2er','PMDPHT','C30CPT1','ACRNtp','CRNtp']

In [42]:
model_removed_copy = model_removed.copy()

Read LP format model from file /var/folders/kn/zzns_smn1q79xdnf1tgqk__r0000gn/T/tmp0tvt1k3c.lp
Reading time = 0.01 seconds
: 1897 rows, 4124 columns, 16590 nonzeros


In [43]:
# with model_removed_copy:
for x in reactions:

    # 'ALDD2ym','C30CPT1','C3STKR1er','C3STKR2er','DNTPPA','DRTPPD','PMDPHT',
    #   'ACRNtp','CRNtp','FA160COAabcp','H2Otm']:
    r = rto.reactions.get_by_id(x).copy()
    model_removed_copy.add_reactions([r])
sol = model_removed_copy.optimize()
if sol.status!='optimal':
    print('infeasible')
else:
    print(sol.objective_value)

0.0613065722993276


In [44]:
model_removed_copy.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,5.101E-05,0,0.00%
cu2_e,EX_cu2_e,3.219E-05,0,0.00%
fe3_e,EX_fe3_e,0.0003782,0,0.00%
glc__D_e,EX_glc__D_e,1,6,98.86%
k_e,EX_k_e,0.03591,0,0.00%
mg2_e,EX_mg2_e,0.003784,0,0.00%
mn2_e,EX_mn2_e,3.721E-05,0,0.00%
na1_e,EX_na1_e,0.0016,0,0.00%
nh4_e,EX_nh4_e,0.4046,0,0.00%
o2_e,EX_o2_e,2.898,0,0.00%


In [45]:
model_removed.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,0.732,6,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-0.732,2,33.33%
btd_RR_e,EX_btd_RR_e,-0.244,4,22.22%
co2_e,EX_co2_e,-0.488,1,11.11%
h_e,EX_h_e,-0.732,0,0.00%
sbt__D_e,EX_sbt__D_e,-0.244,6,33.33%


In [46]:
cobra.io.save_json_model(model_removed_copy,'Lst_v_0.0_removedRTOGenes_forPUB.json')